In [1]:
from qiskit.compiler import transpile, assemble
from helper_fun import get_evaluator_info, evaluate_circ, apply_measurement
from qcg.generators import gen_supremacy, gen_hwea
from qiskit import Aer, IBMQ, execute

device = Aer.get_backend('qasm_simulator')
circ = gen_supremacy(2,2,8)
device_name = 'ibmq_boeblingen'
fields = ['num_shots']
evaluator_info = get_evaluator_info(circ,device_name,fields)
qc = apply_measurement(circ)
qc_transpiled = transpile(qc, backend=device)

qcs = [qc_transpiled for i in range(2)]
qobj = assemble(qcs, backend=device, shots=evaluator_info['num_shots'])
job = device.run(qobj)
result_0 = job.result()
result_1 = job.result()

Generating 2x2, 1+8+1 supremacy circuit


In [3]:
dict_0 = result_0.to_dict()
[print(x, dict_0[x]) for x in dict_0]

qobj_id c4f06d9a-0727-4a00-910a-c59b37ff99d9
job_id b6849b3b-3adc-404d-9f8e-6457551e6b82
backend_name qasm_simulator
success True
results [{'success': True, 'status': 'DONE', 'header': {'qubit_labels': [['q0', 0], ['q0', 1], ['q0', 2], ['q0', 3]], 'clbit_labels': [['c', 0], ['c', 1], ['c', 2], ['c', 3]], 'creg_sizes': [['c', 4]], 'memory_slots': 4, 'name': 'circuit9', 'n_qubits': 4, 'qreg_sizes': [['q0', 4]]}, 'shots': 2048, 'meas_level': 2, 'data': {'counts': {'0xf': 126, '0x1': 53, '0x9': 228, '0x6': 299, '0x4': 4, '0x8': 53, '0xe': 181, '0xb': 81, '0xc': 40, '0x2': 38, '0x3': 145, '0xd': 191, '0x0': 286, '0x5': 160, '0xa': 117, '0x7': 46}}, 'seed_simulator': 465875011, 'time_taken': 0.003984818, 'metadata': {'measure_sampling': True, 'method': 'statevector', 'parallel_shots': 1, 'parallel_state_update': 16}}, {'success': True, 'status': 'DONE', 'header': {'qubit_labels': [['q0', 0], ['q0', 1], ['q0', 2], ['q0', 3]], 'clbit_labels': [['c', 0], ['c', 1], ['c', 2], ['c', 3]], 'creg_siz

[None, None, None, None, None, None, None, None, None, None, None]